In [47]:
import requests
import random
import json
import time
import re
requests.packages.urllib3.disable_warnings()

UA="Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Mobile Safari/537.36"
user_cookie={
"mengpanfei":"_s0=eNpFkMtu00AUhhcIVISoyhugqqsqMTO2ZzyTFYnjNo5rW6kNKV5gfBm7Jh478TjNpULiIVjABhaseSyQeAuC0qq7T9%2F5j36dMzoSTIiirsKlYE04YxsdvEpqLkVlsVlWUlrzqKikuhJRyaQVi6XdkNeV5O3XzCqrvx7sw9b8Dsz0cA9OkcyqiLPDe%2B%2FvBGuf3nWa6UHbLJmzSzxmu57yCa%2FjomQfjqBCZUwI1mSIiKopj37%2F%2BvHny%2Be%2FP78%2FEPhGxKnto3ZcKiE8OVnZ%2BCJ3V2RWGuVsw4O2MS0%2B9FMxoetiQD4adkPpNmYt5FlInHn9Jhy4%2Fkq%2F9ge1Y8NgzFiaWXS7flfFDrquhViYgd7gVvPO15dFzmU9VrPzSE8HwdTc8jwdLbC4ugomuWqdTZfZ5mbO3p4CrgPwMiWQajjBSI0SFSoZAVDW1DgBVIlhlpGHM15wVuXzqMpY8Xqx%2BP%2FeZxBBCAABAMHnnuF5puuEtjs0LsD72yLtHSOIMSYa7I4Caox921V9RCZD7PTN6VTpnvXxpeP2ra5PfOocd5K2BxEFSJUhlZGCOlFyLzCmFJAOX%2Ffgru5T8A%2FdyLJB; Path=/; HttpOnly",
}

def get_httpheader(username="mengpanfei",refer='https://mi.aliyun.com/'):
    if username=="random": 
        username=random.choice(list(user_cookie.keys()))
    if username=="UA": username="mengpanfei"
    Cookie=user_cookie.get(username)
    headers={'Cookie':Cookie,'Referer':refer,'User-Agent':UA,'accept':'application/json',
             'Pragma':'no-cache','Cache-Control':'no-cache','content-type':'application/json'}
    return headers




def get_url_data(url,param,headers,MaxTry=2):
    r=None
    while MaxTry>0:
        try:
            r=requests.post(url,data=param,headers=headers,verify=False)
            if r.json()['errorNo'] in ('00',"30001","30008"): #30001狗状态改变不能操作，30008下架太频繁不能操作
                break
        except Exception as ex:
            print(ex)
            pass
        MaxTry-=1
        time.sleep(0.54)
    return r

def get_selling_domain(suffix="com"):
    headers=get_httpheader("mengpanfei")
    token="tdomain-aliyun-com:fsnDNnSvJzVWZ3sfhBVDsPzfJzrYaRp8"

    domains=[]
    suffix="com"
    minPrice=1
    maxPrice=10
    currentPage=1
    totalCount=500
    while True:
        if currentPage * 500 > totalCount: break
        url="https://domainapi.aliyun.com/onsale/search?fetchSearchTotal=true&token={token}&currentPage={currentPage}&pageSize=500&suffix={suffix}&minPrice={minPrice}&maxPrice={maxPrice}&searchIntro=false&keywordAsPrefix=false&keywordAsSuffix=false&exKeywordAsPrefix=false&exKeywordAsSuffix=false&exKeywordAsPrefix2=false&exKeywordAsSuffix2=false&callback=jQuery111106068888000014647_{_}&_={_}"\
            .format(token=token,currentPage=currentPage,suffix=suffix,minPrice=minPrice,maxPrice=maxPrice,_=int(time.time() * 1000)) # _=int(time.time() * 1000)
        r=requests.get(url,headers=headers,verify=False)
#         print(r.text)
        jr=json.loads(re.findall("\((.+)\)",r.text)[0])
        totalCount=jr["data"]["searchTotal"]
        currentPage+=1
        for domain in jr["data"]["pageResult"]["data"]:
            domains.append(domain)
    return domains

In [48]:
domains=get_selling_domain()

## productType: 13=议价域名, 2=一口价, 4=竞价
for dom in domains:
    if dom["productType"]==13: continue
    if not re.match("[a-z0-9\.]",dom["domainName"]): continue
    price="-"
    if "price" in dom:
        price=dom["price"]
    if int(price)>2: continue #只要1块钱的
    print(dom["deadDate"],dom["productType"],price,dom["domainName"],dom["endTimeRemainingDay"])
    

2020-08-03 4 1 chaodaka.com 7小时27分钟
2020-12-23 4 1 dilushenshi.com 7小时27分钟
2021-01-07 4 1 hiaichain.com 7小时27分钟
2021-01-07 4 1 aichainapp.com 7小时27分钟
2020-11-19 4 1 yaotmall.com 7小时27分钟
2021-03-11 4 1 xwicloud.com 7小时27分钟
2021-04-13 4 2 huibaigou.com 7小时27分钟
2020-07-31 4 1 iosshouzhuan.com 8小时56分钟
2021-03-31 4 1 gtsts.com 1天2小时37分钟
2020-07-03 4 1 bjjzx.com 1天2小时37分钟
2021-05-27 4 1 4swang.com 1天3小时53分钟
2021-06-05 4 1 lriy.com 1天5小时39分钟
2021-03-04 4 1 qdxe.com 1天5小时39分钟
2021-08-22 4 1 yzpa.com 1天5小时39分钟
2021-02-05 4 1 niubozhu.com 1天7小时57分钟
2021-02-22 4 1 jingtengyun.com 1天7小时57分钟
2020-11-13 4 1 jiedehua.com 1天7小时57分钟
2020-12-13 4 1 hongtengge.com 1天7小时57分钟
2020-10-26 4 1 hanghanglian.com 1天7小时57分钟
2020-11-14 2 1 chjfu.com 长期
2020-11-12 2 1 5elicai.com 长期
2020-11-16 2 1 6faka.com 长期
2020-11-16 2 1 100jiyi.com 长期
2020-11-28 2 1 gdylink.com 长期
